# XPECTO Epidemic 2.0: Strategy Examples

This notebook contains a collection of intervention strategies for managing epidemics in the XPECTO competition, ranging from simple to advanced. Each approach is explained step by step to help you understand how to develop effective strategies.

## Setup

First, let's set up our environment and import the necessary modules.

In [ ]:
# Add the project root to the path
import sys
import os
from pathlib import Path

# Add parent directory to path for imports
sys.path.append(str(Path.cwd().parent))

# Import required modules
from src.competition import CompetitionManager
from src.competition.testing.enhanced_engine import EnhancedEngine as Engine

# Import visualization libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import src.competition.utils as utils

# Set up visualization
plt.style.use('seaborn-whitegrid')
sns.set(font_scale=1.2)
pd.set_option('display.max_columns', None)

## Initialize Competition System

Set up the competition environment in practice mode to test our strategies.

In [ ]:
# Create an epidemic engine
engine = Engine()

# Create competition manager with a practice data directory
competition = CompetitionManager(data_dir="practice_data", engine=engine)

# Register as a player
player_name = "Your Name"  # Change this to your name
player_id = competition.setup_player(name=player_name)
print(f"Registered as player: {player_name} (ID: {player_id})")

# Ensure we're in practice mode
competition.toggle_practice_mode(is_practice=True)
print("Practice mode enabled - attempts will not count for competition")

## Available Engine Controls

Before we dive into specific strategies, let's review the key control methods provided by the engine:

1. **Lockdown Control**
   - `engine.set_lockdown_level(level)` - Sets lockdown severity (0.0-1.0)
   
2. **Resource Allocation**
   - `engine.allocate_resources(category, amount)` - Allocates resources to categories like healthcare, economic, research, testing
   
3. **Travel Restrictions**
   - `engine.restrict_travel(should_restrict)` - Enables/disables travel restrictions
   
4. **Regional Controls** (Enhanced Engine only)
   - `engine.set_regional_lockdown(region, level)` - Region-specific lockdown
   - `engine.allocate_regional_resources(region, category, amount)` - Region-specific resource allocation
   
5. **Monitoring & Adaptation**
   - `engine.register_step_callback(callback_function)` - Registers a function that runs at each simulation step

## 1. Basic Strategies

### 1.1 Fixed Approach Strategy

The simplest approach is to set fixed intervention levels at the beginning and maintain them throughout the simulation.

In [ ]:
def fixed_strategy(engine):
    """A simple strategy that applies fixed measures throughout the simulation."""
    
    # Set moderate lockdown
    engine.set_lockdown_level(0.5)  # 50% lockdown severity
    
    # Allocate resources to key areas
    engine.allocate_resources('healthcare', 300)  # Strengthen healthcare system
    engine.allocate_resources('research', 200)    # Invest in research for vaccines/treatments
    engine.allocate_resources('economic', 100)    # Provide some economic support
    
    # Apply travel restrictions
    if hasattr(engine, 'restrict_travel'):
        engine.restrict_travel(True)

In [ ]:
# Set up and run the simulation with our fixed strategy
competition.set_scenario("standard")
competition.setup_simulation()
fixed_results = competition.run_simulation(steps=365, interventions=[fixed_strategy])

# Display results
competition.display_score(fixed_results)

### 1.2 Phased Strategy

A more effective approach is to divide the epidemic into phases and apply different measures in each phase.

In [ ]:
def phased_strategy(engine):
    """A strategy that applies different measures in predefined phases."""
    
    # Register a callback to implement phased approach
    def phase_based_response(step, state):
        # Early phase - containment (days 0-60)
        if step < 60:
            engine.set_lockdown_level(0.7)  # Strict initial lockdown to contain spread
            engine.allocate_resources('healthcare', 20)  # Prepare healthcare system
            engine.allocate_resources('research', 15)    # Start research early
            
        # Middle phase - balanced approach (days 60-180)
        elif 60 <= step < 180:
            engine.set_lockdown_level(0.5)  # Moderate lockdown
            engine.allocate_resources('healthcare', 15)
            engine.allocate_resources('economic', 10)     # Support economy during extended measures
            engine.allocate_resources('research', 10)     # Continue research
            
        # Late phase - recovery (days 180+)
        else:
            engine.set_lockdown_level(0.3)  # Relaxed lockdown
            engine.allocate_resources('economic', 20)     # Focus on economic recovery
            engine.allocate_resources('healthcare', 10)   # Maintain healthcare support
    
    # Initial measures
    engine.set_lockdown_level(0.7)
    engine.allocate_resources('healthcare', 200)  # Initial healthcare investment
    engine.allocate_resources('research', 100)    # Initial research investment
    
    # Apply travel restrictions
    if hasattr(engine, 'restrict_travel'):
        engine.restrict_travel(True)
    
    # Register our callback
    engine.register_step_callback(phase_based_response)

In [ ]:
# Run the phased strategy simulation
competition.set_scenario("standard")
competition.setup_simulation()
phased_results = competition.run_simulation(steps=365, interventions=[phased_strategy])

# Display results
competition.display_score(phased_results)

## 2. Intermediate Strategies

### 2.1 Adaptive Threshold Strategy

This strategy adapts measures based on the current infection rate, using thresholds to determine appropriate interventions.

In [ ]:
def adaptive_threshold_strategy(engine):
    """A strategy that adapts measures based on infection rate thresholds."""
    
    # Initial measures
    engine.set_lockdown_level(0.4)  # Start with moderate lockdown
    engine.allocate_resources('healthcare', 200)
    engine.allocate_resources('research', 100)
    
    # Track state across steps
    state_tracker = {
        'prev_infection_rate': 0.0,
        'trend_increasing': False,
        'days_above_threshold': 0,
        'days_below_threshold': 0
    }
    
    def threshold_based_response(step, state):
        # Calculate current infection rate
        infection_rate = state.population.infected / state.population.total
        
        # Track infection trend
        state_tracker['trend_increasing'] = infection_rate > state_tracker['prev_infection_rate']
        state_tracker['prev_infection_rate'] = infection_rate
        
        # Update threshold counters
        if infection_rate > 0.1:  # High infection threshold
            state_tracker['days_above_threshold'] += 1
            state_tracker['days_below_threshold'] = 0
        elif infection_rate < 0.05:  # Low infection threshold
            state_tracker['days_below_threshold'] += 1
            state_tracker['days_above_threshold'] = 0
        
        # Apply different strategies based on thresholds
        
        # Crisis response - high infection rate
        if infection_rate > 0.15:
            engine.set_lockdown_level(0.8)  # Severe lockdown
            engine.allocate_resources('healthcare', 30)  # Surge healthcare resources
            if hasattr(engine, 'restrict_travel'):
                engine.restrict_travel(True)  # Restrict travel
                
        # High response - sustained high infection
        elif infection_rate > 0.1 or state_tracker['days_above_threshold'] > 14:
            engine.set_lockdown_level(0.7)  # Strict lockdown
            engine.allocate_resources('healthcare', 20)
            engine.allocate_resources('economic', 10)  # Economic support during strict measures
            if hasattr(engine, 'restrict_travel'):
                engine.restrict_travel(True)
                
        # Medium response - moderate infection or increasing trend
        elif infection_rate > 0.05 or state_tracker['trend_increasing']:
            engine.set_lockdown_level(0.5)  # Moderate lockdown
            engine.allocate_resources('healthcare', 15)
            engine.allocate_resources('research', 10)
            if hasattr(engine, 'restrict_travel'):
                engine.restrict_travel(infection_rate > 0.07)
                
        # Low response - sustained low infection
        elif state_tracker['days_below_threshold'] > 21:
            engine.set_lockdown_level(0.2)  # Minimal restrictions
            engine.allocate_resources('economic', 20)  # Focus on economic recovery
            engine.allocate_resources('healthcare', 5)  # Maintain healthcare
            if hasattr(engine, 'restrict_travel'):
                engine.restrict_travel(False)
                
        # Default response
        else:
            engine.set_lockdown_level(0.4)  # Moderate lockdown
            engine.allocate_resources('healthcare', 10)
            engine.allocate_resources('research', 5)
            engine.allocate_resources('economic', 5)
    
    # Register callback
    engine.register_step_callback(threshold_based_response)

In [ ]:
# Run the adaptive threshold strategy simulation
competition.set_scenario("standard")
competition.setup_simulation()
threshold_results = competition.run_simulation(steps=365, interventions=[adaptive_threshold_strategy])

# Display results
competition.display_score(threshold_results)